In [1]:
! pip install snowflake-ml-python==1.0.2

     |████████████████████████████████| 1.9MB 8.2MB/s eta 0:00:01
     |████████████████████████████████| 4.2MB 56.9MB/s eta 0:00:01
     |████████████████████████████████| 133kB 109.6MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 98.5MB/s eta 0:00:01
     |████████████████████████████████| 24.6MB 89.6MB/s eta 0:00:01
     |████████████████████████████████| 737kB 65.4MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 26kB/s s eta 0:00:01
     |████████████████████████████████| 51kB 4.5MB/s  eta 0:00:01
     |████████████████████████████████| 17.3MB 99.2MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 87.7MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 82.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.4MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 5.1MB/s  eta 0:00:01
     |████████████████████████████████| 327kB 78.7MB/s eta 0:00:01
     |████████████████████████████████| 143kB 98.0MB/s eta

     |████████████████████████████████| 61kB 3.6MB/s  eta 0:00:01
     |████████████████████████████████| 204kB 88.4MB/s eta 0:00:01
     |████████████████████████████████| 163kB 101.1MB/s eta 0:00:01
     |████████████████████████████████| 36.0MB 121kB/s  eta 0:00:01
     |████████████████████████████████| 307kB 63kB/s /s eta 0:00:01
     |████████████████████████████████| 71kB 6.2MB/s  eta 0:00:01
     |████████████████████████████████| 808kB 90.0MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 47.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 106.2MB/s eta 0:00:01
     |████████████████████████████████| 122kB 96.4MB/s eta 0:00:01
     |████████████████████████████████| 61kB 4.1MB/s  eta 0:00:01
     |████████████████████████████████| 276kB 77.3MB/s eta 0:00:01
     |████████████████████████████████| 225kB 95.9MB/s eta 0:00:01
ERROR: snowflake-snowpark-python 1.8.0 has requirement cloudpickle<=2.0.0,>=1.6.0, but you'll have cloudpickle 2.2.1 which is in

In [1]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

/tmp/pip_packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (10.0.1), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [3]:
session = get_session()

# Model Building
### Load the final data from snowflake table

In [4]:
table_name = 'predictive_maintenance_final'

In [5]:
sf_df = session.table(table_name).drop('ROW')

In [6]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

In [8]:
a = sf_df.to_pandas()

In [11]:
list(a.columns)

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

In [10]:
a.head()

,FAILURE,METRIC5,METRIC6,DAY,DAY_WEEK,SECTOR,OP_PERIOD,DEV_RECONNECTED,MNW1,DIF_M6,DIF_M5,LOG_M2,LOG_M3,LOG_M4,LOG_M7,LOG_M9
0,0,6,407438,1,3,S1F0,1,0,26953834,0,0,2.079442,0.000000,3.970292,0.0,2.079442
1,0,6,403174,1,3,S1F0,1,0,7671335,0,0,0.000000,1.386294,0.000000,0.0,0.000000
2,0,12,237394,1,3,S1F0,1,0,21661996,0,0,0.000000,0.000000,0.000000,0.0,0.000000
3,0,6,410186,1,3,S1F0,1,0,9961753,0,0,0.000000,0.000000,0.000000,0.0,0.000000
4,0,15,313173,1,3,S1F0,1,0,16996310,0,0,0.000000,0.000000,0.000000,0.0,1.386294


In [13]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [15]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [16]:
train_df.show(3)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"  |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0       |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |12         |237394     |1      |3           |S1F0      |1            |0                  |21661996  |0     

Create a pipeline with preprocessing steps and model definition

In [14]:
pipeline = Pipeline(
    steps=[
            (
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                XGBClassifier(
                input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS + ["DEV_RECONNECTED"],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [18]:
pipeline.fit(a)

/tmp/pip_packages/sklearn/base.py:457: UserWarning: X has feature names, but OrdinalEncoder was fitted without feature names
  warnings.warn(


AttributeError: 'OrdinalEncoder' object has no attribute '_infrequent_enabled'

In [ ]:
result = pipeline.predict(test_df)